In [ ]:
from glob import glob
import os
import itertools
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.axes_grid1 import ImageGrid, make_axes_locatable, host_subplot
import matplotlib.ticker as ticker
from matplotlib.collections import Collection, LineCollection
from matplotlib.artist import allow_rasterization
from matplotlib.font_manager import FontProperties
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.dates as md
import matplotlib.ticker as ticker
from metpy.plots import ctables
#from pyPIPS import disdrometer_module as dis
from pyPIPS import pips_io as pipsio
from pyPIPS import timemodule as tm
from pyPIPS.utils import mtokm, getTimeWindow, interp_along_1D
from pyPIPS import plotmodule as pm
from pyPIPS import PIPS as pips
# from pyPIPS import fmcw
from itertools import cycle
import netCDF4 as netcdf
from datetime import datetime, timedelta
import pandas as pd
import xarray as xr
#from skimage.restoration import unwrap_phase
#import skimage.feature as SF
import numpy.ma as ma
from scipy import ndimage, interpolate
import pyart
import cartopy.crs as ccrs
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
IOP_name = 'IOP2'
radar_name = 'KGWX'
radar_type= 'NEXRAD'
date = '0330'

gridded_output_dir = f'/Users/dawson29/Projects/PERiLS/obsdata/2022/NEXRAD/GRID3/{IOP_name}/{radar_name}'
if not os.path.exists(gridded_output_dir):
    os.makedirs(gridded_output_dir)
plot_dir = f'/Users/dawson29/Projects/PERiLS/obsdata/2022/NEXRAD/GRID3/{IOP_name}/{radar_name}/plots/'
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)

radar_starttimestamp = '20220330225909'
radar_endtimestamp = '20220330225909'

radar_startdatetime = datetime.strptime(radar_starttimestamp, '%Y%m%d%H%M%S')
radar_enddatetime = datetime.strptime(radar_endtimestamp, '%Y%m%d%H%M%S')

radar_start_timestamp = radar_startdatetime.strftime('%Y%m%d%H%M')
radar_end_timestamp = radar_enddatetime.strftime('%Y%m%d%H%M')

cfradial_dir = \
    f'/Users/dawson29/sshfs_mounts/depot/data/Projects/PERiLS/obsdata/2022/NEXRAD/{IOP_name}/{radar_name}'
cfradial_files = sorted(glob(cfradial_dir + '/*V06.nc'))

cfradial_files_to_read = []
for cfradial_file in cfradial_files:
    radar_file_timestamp = os.path.basename(cfradial_file)[4:19]
    radar_file_datetime = datetime.strptime(radar_file_timestamp, '%Y%m%d_%H%M%S')
    if radar_file_datetime >= radar_startdatetime and radar_file_datetime <= radar_enddatetime:
        cfradial_files_to_read.append(cfradial_file)
        print(os.path.basename(cfradial_file))

In [ ]:
# Read the first file for testing
radarobj = pyart.io.read(cfradial_files_to_read[0])

In [ ]:
radarobj = radarobj.extract_sweeps([0, 1, 3, 4, 5, 7, 8, 9, 10, 12, 13, 14, 15, 16])

In [ ]:
radarobj.fixed_angle['data']

In [ ]:
# messing around with smoothing ZDR
# Taken from an old PyART short course notebook here:
# https://notebook.community/openradar/pyart_short_course/2%20Simple%20Py-ART%20Usage%20plotting%20PPI%20data%20on%20a%20map%20and%20add%20a%20new%20field

smooth_zdr = radarobj.fields['ZDR']['data'].copy()
# Now for each of the radials in the volume we want to do an 8-point smooth by convolving the data with a 
# Hanning window..
# rather than write out the code here we are going to use Py-ART's smooth and trim function which also makes sure
# the returned array has the same size as the original array.. 
# Check here: https://github.com/ARM-DOE/pyart/blob/master/pyart/correct/phase_proc.py#L242

for i in range(smooth_zdr.shape[0]):
    smooth_zdr[i,:] = pyart.correct.phase_proc.smooth_and_trim(smooth_zdr[i,:], 8)

# Now that we have add this data as a new field, using the same metadata as the 
# differential_reflectivity field.

radarobj.add_field_like('ZDR', 'ZDR_smooth',  smooth_zdr, replace_existing=True)

In [ ]:
# messing around with smoothing ZDR. Try a 1-D 5-point median filter along rays

footprint = [[1, 1, 1, 1, 1]]

median_zdr = radarobj.fields['ZDR']['data'].copy()

print(median_zdr.shape)

median_zdr = ndimage.median_filter(median_zdr, footprint=footprint)
radarobj.add_field_like('ZDR', 'ZDR_median',  median_zdr, replace_existing=True)

# for i in range(smooth_zdr.shape[0]):
#     smooth_zdr[i,:] = pyart.correct.phase_proc.smooth_and_trim(smooth_zdr[i,:], 8)

# # Now that we have add this data as a new field, using the same metadata as the 
# # differential_reflectivity field.

# radarobj.add_field_like('ZDR', 'ZDR_smooth',  smooth_zdr)

In [ ]:
# Set up grid limits and spacing

xmin = -123000.
xmax = 123000.
dx = 500.
nx = int((xmax - xmin) / dx) + 1

ymin = -123000.
ymax = 123000.
dy = 500.
ny = int((ymax - ymin) / dy) + 1

zmin = 500.
zmax = 4500.
dz = 250.
nz = int((zmax - zmin) / dz) + 1
print(nz)

In [ ]:
# Test grid first radar file
grid = pyart.map.grid_from_radars(
    (radarobj,),
    grid_shape=(nz, ny, nx),
    grid_limits=((zmin, zmax), (ymin, ymax), (xmin, xmax)),
    fields=['REF', 'ZDR', 'ZDR_smooth', 'ZDR_median', 'RHO'],
    weighting_function='Barnes2',
    map_roi=True)

grid_xr = grid.to_xarray()
print(grid_xr)

In [ ]:
grid_display = pyart.graph.GridMapDisplay(grid)
# Setting projection, figure size, and panel sizes.
projection = ccrs.PlateCarree()

lat = 33.7
lon = -88.4
vmin = -10
vmax = 65

fig_grid, ax_grid = plt.subplots(figsize=(10, 10), subplot_kw={'projection': projection})
grid_display.plot_grid('REF', 0, vmin=vmin, vmax=vmax,
                       ax=ax_grid,
                       projection=projection,
                       cmap='pyart_NWSRef')
grid_display.plot_crosshairs(lon=lon, lat=lat)

# Get the bounds of the axes in the axes coordinate system
bounds = ax_grid.get_extent()
print(bounds)

In [ ]:
lat = 33.7
lon = -88.4
vmin = -1
vmax = 8

fig_grid, ax_grid = plt.subplots(figsize=(10, 10), subplot_kw={'projection': projection})
grid_display.plot_grid('ZDR', 0, vmin=vmin, vmax=vmax,
                       ax=ax_grid,
                       projection=projection,
                       cmap='pyart_RefDiff')
grid_display.plot_crosshairs(lon=lon, lat=lat)

In [ ]:
lat = 33.7
lon = -88.4
vmin = -1
vmax = 8

fig_grid, ax_grid = plt.subplots(figsize=(10, 10), subplot_kw={'projection': projection})
grid_display.plot_grid('ZDR_median', 0, vmin=vmin, vmax=vmax,
                       ax=ax_grid,
                       projection=projection,
                       cmap='pyart_RefDiff')
grid_display.plot_crosshairs(lon=lon, lat=lat)

In [ ]:
lat = 33.7
lon = -88.4
vmin = -1
vmax = 8

fig_grid, ax_grid = plt.subplots(figsize=(10, 10), subplot_kw={'projection': projection})
grid_display.plot_grid('ZDR_smooth', 0, vmin=vmin, vmax=vmax,
                       ax=ax_grid,
                       projection=projection,
                       cmap='pyart_RefDiff')
grid_display.plot_crosshairs(lon=lon, lat=lat)

In [ ]:
lat = 33.7
lon = -88.4
vmin = -1
vmax = 8

fig_grid, ax_grid = plt.subplots(figsize=(10, 10), subplot_kw={'projection': projection})
grid_display.plot_grid('ROI', 5,
                       ax=ax_grid,
                       projection=projection,
                       cmap='pyart_Bu10')
grid_display.plot_crosshairs(lon=lon, lat=lat)

In [ ]:
ppi_display = pyart.graph.RadarMapDisplay(radarobj)

In [ ]:
projection = ccrs.PlateCarree()
fig, ax = plt.subplots(figsize=[10, 10], subplot_kw={'projection': projection})
ppi_display.plot_ppi_map(field='REF', sweep=0, vmin=-10, vmax = 65, cmap='pyart_NWSRef', ax=ax)
ppi_display.plot_range_rings([100, 200, 300, 400], lw=1)
ax.set_extent(bounds)
bounds2 = ax.get_extent()
print(bounds2) # should be the same as original bounds

In [ ]:
projection = ccrs.PlateCarree()
fig, ax = plt.subplots(figsize=[10, 10], subplot_kw={'projection': projection})
ppi_display.plot_ppi_map(field='ZDR', sweep=0, vmin=-1, vmax = 8, cmap='pyart_RefDiff', ax=ax)
ppi_display.plot_range_rings([100, 200, 300, 400], lw=1)
ax.set_extent(bounds)
bounds2 = ax.get_extent()
print(bounds2) # should be the same as original bounds

In [ ]:
projection = ccrs.PlateCarree()
fig, ax = plt.subplots(figsize=[10, 10], subplot_kw={'projection': projection})
ppi_display.plot_ppi_map(field='ZDR_smooth', sweep=0, vmin=-1, vmax = 8, cmap='pyart_RefDiff', ax=ax)
ppi_display.plot_range_rings([100, 200, 300, 400], lw=1)
ax.set_extent(bounds)
bounds2 = ax.get_extent()
print(bounds2) # should be the same as original bounds

In [ ]:
projection = ccrs.PlateCarree()
fig, ax = plt.subplots(figsize=[10, 10], subplot_kw={'projection': projection})
ppi_display.plot_ppi_map(field='ZDR_median', sweep=0, vmin=-1, vmax = 8, cmap='pyart_RefDiff', ax=ax)
ppi_display.plot_range_rings([100, 200, 300, 400], lw=1)
ax.set_extent(bounds)
bounds2 = ax.get_extent()
print(bounds2) # should be the same as original bounds

In [ ]:
# Create gridded radar objects and dump to disk
for i, f in enumerate(cfradial_files_to_read):
    print("On file {:d} of {:d} ({:.2f}%)".format(i+1, len(cfradial_files), (i+1)/len(cfradial_files)*100.))
    print("Reading {}".format(os.path.basename(f)))
    radar = pyart.io.read_nexrad_archive(f)
    output_file = os.path.basename(f)[:-5]+"_gridded.nc"
    output_path = os.path.join(gridded_output_dir, output_file)
    
    print("Mapping to grid!")
    grid = pyart.map.grid_from_radars(
        (radar,),
        grid_shape=(nz, ny, nx),
        grid_limits=((zmin, zmax), (ymin, ymax), (xmin, xmax)),
        fields=['differential_reflectivity', 'reflectivity', 'cross_correlation_ratio'],
        weighting_function='Barnes2')
    print("Writing {}".format(output_file))
    pyart.io.write_grid(output_path, grid, arm_time_variables=True, arm_alt_lat_lon_variables=True, 
                        write_point_x_y_z=True, write_point_lon_lat_alt=True)
